In [13]:
import pandas as pd
aflac_df = pd.read_csv("/Users/lukelockhart/Downloads/Data/aflac_tweets_cleaned.csv")
aflac_df.head(5)
import pandas as pd

# Convert to datetime format
aflac_df["tweet_date"] = pd.to_datetime(aflac_df["tweet_date"], format="%m/%d/%Y %H:%M")

# Find the maximum date
max_date = aflac_df["tweet_date"].max()

print(max_date)

2024-11-13 20:55:00


In [35]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

# Function to generate a random delay between actions
def random_delay():
    return random.uniform(5.5, 15.0)  # Shorter delays for faster scraping

def categorize_keyword(keyword):
    """ Assigns the correct bucket based on the keyword. """
    brand_keywords = {
        "Aflac company", "Aflac ads", "Aflac advertisements", "Aflac marketing",
        "Aflac brand", "Aflac commercials", "Aflac social media", "Aflac duck", "Aflac mascot"
    }
    
    experience_keywords = {
        "Aflac reviews", "Aflac testimonials", "Aflac feedback", "Aflac complaints",
        "Aflac service", "Aflac customer service", "Aflac support", "Aflac claims process",
        "Aflac claim review", "Aflac help", "Aflac agents", "Aflac representatives",
        "Aflac customer care", "Aflac call center"
    }
    
    product_keywords = {
        "Aflac policy", "Aflac policies", "Aflac insurance", "Aflac benefits",
        "Aflac coverage", "Aflac supplemental insurance", "Aflac accident insurance",
        "Aflac health insurance", "Aflac dental insurance", "Aflac cancer policy",
        "Aflac life insurance", "Aflac short-term disability", "Aflac products",
        "Aflac plans", "Aflac rates", "Aflac enrollment", "Aflac payment"
    }
    
    if keyword in brand_keywords:
        return "Brand"
    elif keyword in experience_keywords:
        return "Experience"
    elif keyword in product_keywords:
        return "Products"
    else:
        return "Unknown"

def login_and_scrape_links(username, password, keywords, cutoff_date_str):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    links_data = []  # List to store keyword, link, text, date, and bucket
    cutoff_date = datetime.strptime(cutoff_date_str, "%Y-%m-%d")

    try:
        # Open the login page
        driver.get("https://x.com/login")
        
        # Wait for manual login
        input("Log in manually, then press Enter to start scraping...")

        # Verify login was successful
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='AppTabBar_Home_Link']"))
        )

        # Loop through all keywords
        for keyword in keywords:
            bucket = categorize_keyword(keyword)  # Get bucket category
            print(f"Collecting tweets for keyword: {keyword} (Bucket: {bucket})")

            search_url = f"https://x.com/search?q={keyword}&src=typed_query&f=live"
            driver.get(search_url)
            time.sleep(random_delay())  # Shorter delay for speed

            last_height = driver.execute_script("return document.body.scrollHeight")
            stop_scrolling = False

            while not stop_scrolling:
                tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
                for tweet in tweets:
                    try:
                        # Extract tweet URL
                        link_element = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/')]")
                        link = link_element.get_attribute('href')

                        # Extract tweet date
                        date_element = tweet.find_element(By.XPATH, ".//time")
                        tweet_date = datetime.fromisoformat(date_element.get_attribute('datetime')[:-1])

                        # Extract tweet text
                        text_elements = tweet.find_elements(By.XPATH, ".//div[@data-testid='tweetText']")
                        tweet_text = " ".join([elem.text for elem in text_elements]).strip() if text_elements else ""

                        if tweet_date >= cutoff_date:
                            links_data.append((keyword, link, tweet_text, tweet_date, bucket))
                            print(f"Scraped: {link} - {tweet_date} - {bucket}")
                            print(f"Tweet Text: {tweet_text}\n")
                        else:
                            print(f"Tweet from {tweet_date} is older than cutoff. Stopping scrolling.")
                            stop_scrolling = True
                            break  # Stop collecting for this keyword

                    except Exception as e:
                        print(f"Error processing tweet: {e}")
                        continue

                if stop_scrolling:
                    break  # Stop scrolling if tweets are too old

                # Scroll down and wait (optimized scrolling)
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
                time.sleep(random_delay())

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # Stop if no new content loads
                last_height = new_height

    except Exception as e:
        print(f"Error while collecting links: {e}")

    finally:
        driver.quit()

    return links_data

# Usage
username = "your_username_here"  # You do not need to replace anything as you will manually log in
password = "your_password_here"  # Replace with your actual password
keywords = [
    # Brand
    "Aflac company", "Aflac ads", "Aflac advertisements", "Aflac marketing", 
    "Aflac brand", "Aflac commercials", "Aflac social media", "Aflac duck", "Aflac mascot",
    # Experience
    "Aflac reviews", "Aflac testimonials", "Aflac feedback", "Aflac complaints", 
    "Aflac service", "Aflac customer service", "Aflac support", "Aflac claims process", 
    "Aflac claim review", "Aflac help", "Aflac agents", "Aflac representatives", 
    "Aflac customer care", "Aflac call center",
    # Products
    "Aflac policy", "Aflac policies", "Aflac insurance", "Aflac benefits", 
    "Aflac coverage", "Aflac supplemental insurance", "Aflac accident insurance", 
    "Aflac health insurance", "Aflac dental insurance", "Aflac cancer policy", 
    "Aflac life insurance", "Aflac short-term disability", "Aflac products", 
    "Aflac plans", "Aflac rates", "Aflac enrollment", "Aflac payment"
]

# Set the cutoff date manually (YYYY-MM-DD)
cutoff_date_str = "2024-11-13"  # Only scrape tweets after this date

# Run the scraper
all_links_data = login_and_scrape_links(username, password, keywords, cutoff_date_str)

# Save the links to a DataFrame and export to CSV
if all_links_data:
    df_links = pd.DataFrame(all_links_data, columns=["keyword", "tweet_url", "tweet_text", "tweet_date", "bucket"])
else:
    print("No links found.")


Log in manually, then press Enter to start scraping... 


Scraped: https://x.com/claudi_surfing/status/1898331370994147417 - 2025-03-08 11:14:02 - Brand
Tweet Text: Insurance Giant Aflac Reportedly Sued For Worker Exploitation. The company calls the allegations baseless.

Scraped: https://x.com/DarthDividend23/status/1898213138530521099 - 2025-03-08 03:24:13 - Brand
Tweet Text: Pretty good start for the month of March with the Dividendo Manifesto 

Dividends from;

$ENB Enbridge 
$V Visa 
$TGT Target 
$AFL Aflac 
$COP ConocoPhillips 
$KR The Kroger Company

Scraped: https://x.com/darioalvez/status/1898182215328030769 - 2025-03-08 01:21:21 - Brand
Tweet Text: Insurance Giant Aflac Reportedly Sued For Worker Exploitation. The company calls the allegations baseless.

Scraped: https://x.com/HIimbelieber2/status/1897825311410016483 - 2025-03-07 01:43:08 - Brand
Tweet Text: Insurance Giant Aflac Reportedly Sued For Worker Exploitation. The company calls the allegations baseless.

#Harmony

Scraped: https://x.com/JulesMoffit/status/18973874928669821

In [39]:
df_links.head(10)

,keyword,tweet_url,tweet_text,tweet_date,bucket
0,Aflac company,https://x.com/claudi_surfing/status/1898331370...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 11:14:02,Brand
1,Aflac company,https://x.com/DarthDividend23/status/189821313...,Pretty good start for the month of March with ...,2025-03-08 03:24:13,Brand
2,Aflac company,https://x.com/darioalvez/status/18981822153280...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 01:21:21,Brand
3,Aflac company,https://x.com/HIimbelieber2/status/18978253114...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-07 01:43:08,Brand
4,Aflac company,https://x.com/JulesMoffit/status/1897387492866...,You believe what people are spending on drugs ...,2025-03-05 20:43:24,Brand
5,Aflac company,https://x.com/HIimbelieber2/status/18978253114...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-07 01:43:08,Brand
6,Aflac company,https://x.com/JulesMoffit/status/1897387492866...,You believe what people are spending on drugs ...,2025-03-05 20:43:24,Brand
7,Aflac company,https://x.com/Norther55386970/status/189720914...,"Idag arbetsbefriad inkomst från Aflac, Fortis,...",2025-03-05 08:54:42,Brand
8,Aflac company,https://x.com/ricardoalzate4/status/1896875175...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-04 10:47:38,Brand
9,Aflac company,https://x.com/nataly18070327/status/1896147630...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-02 10:36:38,Brand


In [41]:
df_links.to_csv("Aflac_Twitter_Scrape.csv", index = False)

In [13]:
import pandas as pd
df_links = pd.read_csv("/Users/lukelockhart/Aflac_Twitter_Scrape.csv")
df_links

,keyword,tweet_url,tweet_text,tweet_date,bucket
0,Aflac company,https://x.com/claudi_surfing/status/1898331370...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 11:14:02,Brand
1,Aflac company,https://x.com/DarthDividend23/status/189821313...,Pretty good start for the month of March with ...,2025-03-08 03:24:13,Brand
2,Aflac company,https://x.com/darioalvez/status/18981822153280...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 01:21:21,Brand
3,Aflac company,https://x.com/HIimbelieber2/status/18978253114...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-07 01:43:08,Brand
4,Aflac company,https://x.com/JulesMoffit/status/1897387492866...,You believe what people are spending on drugs ...,2025-03-05 20:43:24,Brand
...,...,...,...,...,...
824,Aflac enrollment,https://x.com/csgactuarial/status/188428656737...,Aflac Medicare Supplement is the newest applic...,2025-01-28 17:05:00,Products
825,Aflac enrollment,https://x.com/aflac_babiya/status/187126234716...,Enrollment season is here again. Get yours off...,2024-12-23 18:31:24,Products
826,Aflac enrollment,https://x.com/AflacNick/status/186902379585328...,Benefits planning can lead to some pretty impo...,2024-12-17 14:16:12,Products
827,Aflac enrollment,https://x.com/aflac_babiya/status/186402947498...,The easier you make open enrollment for employ...,2024-12-03 19:30:33,Products


In [5]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

# Function to generate a random delay between actions
def random_delay():
    return random.uniform(5.5, 15.0)  # Shorter delays for faster scraping

def login_and_scrape_experience(username, password, experience_keywords, cutoff_date_str):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    new_data = []  # List to store new Experience tweets
    cutoff_date = datetime.strptime(cutoff_date_str, "%Y-%m-%d")

    try:
        # Open the login page
        driver.get("https://x.com/login")
        
        # Wait for manual login
        input("Log in manually, then press Enter to start scraping...")

        # Verify login was successful
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='AppTabBar_Home_Link']"))
        )

        # Loop through Experience keywords
        for keyword in experience_keywords:
            print(f"Collecting Experience tweets for keyword: {keyword}")

            search_url = f"https://x.com/search?q={keyword}&src=typed_query&f=live"
            driver.get(search_url)
            time.sleep(random_delay())  # Shorter delay for speed

            last_height = driver.execute_script("return document.body.scrollHeight")
            stop_scrolling = False

            while not stop_scrolling:
                tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
                for tweet in tweets:
                    try:
                        # Extract tweet URL
                        link_element = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/')]")
                        link = link_element.get_attribute('href')

                        # Extract tweet date
                        date_element = tweet.find_element(By.XPATH, ".//time")
                        tweet_date = datetime.fromisoformat(date_element.get_attribute('datetime')[:-1])

                        # Extract tweet text
                        text_elements = tweet.find_elements(By.XPATH, ".//div[@data-testid='tweetText']")
                        tweet_text = " ".join([elem.text for elem in text_elements]).strip() if text_elements else ""

                        if tweet_date >= cutoff_date:
                            new_data.append(("Experience", keyword, link, tweet_text, tweet_date))
                            print(f"Scraped: {link} - {tweet_date}")
                            print(f"Tweet Text: {tweet_text}\n")
                        else:
                            print(f"Tweet from {tweet_date} is older than cutoff. Stopping scrolling.")
                            stop_scrolling = True
                            break  # Stop collecting for this keyword

                    except Exception as e:
                        print(f"Error processing tweet: {e}")
                        continue

                if stop_scrolling:
                    break  # Stop scrolling if tweets are too old

                # Scroll down and wait (optimized scrolling)
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
                time.sleep(random_delay())

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # Stop if no new content loads
                last_height = new_height

    except Exception as e:
        print(f"Error while collecting links: {e}")

    finally:
        driver.quit()

    return new_data

# Experience keywords (includes new ones for better results)
experience_keywords = [
    "Aflac customer service issues", "Aflac customer support wait time",
    "Aflac agent rude", "Aflac claim process slow", "Aflac not responding",
    "Aflac denied my claim", "Aflac late payment", "Aflac claim approval time",
    "Aflac claim rejected", "Aflac claim status", "Aflac sucks", "Aflac is a scam",
    "Aflac is terrible", "Aflac review", "Aflac worst experience",
    "Aflac helped me", "Aflac great service", "Aflac paid my claim fast",
    "Aflac saved me money", "Aflac experience"
]

# Set the cutoff date manually (YYYY-MM-DD)
cutoff_date_str = "2024-11-13"  # Only scrape tweets after this date

# Run the scraper for Experience tweets
new_experience_data = login_and_scrape_experience("your_username_here", "your_password_here", experience_keywords, cutoff_date_str)

# Convert new data to DataFrame
df_new = pd.DataFrame(new_experience_data, columns=["bucket", "keyword", "tweet_url", "tweet_text", "tweet_date"])
df_new

Log in manually, then press Enter to start scraping... 


Scraped: https://x.com/cnbroussard/status/1882870498834669833 - 2025-01-24 19:18:03
Tweet Text: @Entergy
 I'm having customer service issues, I need help. Can you dm me?

Tweet from 2024-05-17 16:31:59 is older than cutoff. Stopping scrolling.
Tweet from 2015-04-24 17:49:18 is older than cutoff. Stopping scrolling.
Tweet from 2019-12-06 15:32:52 is older than cutoff. Stopping scrolling.
Tweet from 2022-01-15 23:18:15 is older than cutoff. Stopping scrolling.
Scraped: https://x.com/ludovico_carlo5/status/1897626926031733037 - 2025-03-06 12:34:50
Tweet Text: @aflac had my Hospital Indemnity claim denied unfairly. Horrible way to treat clients.

Scraped: https://x.com/ludovico_carlo5/status/1897476648531878155 - 2025-03-06 02:37:41
Tweet Text: @aflac
 I was just denied my Hospital indemnity insurance claim unfairly. Poor service and even worse way to treat your clients.

Tweet from 2024-10-02 02:54:12 is older than cutoff. Stopping scrolling.
Tweet from 2023-06-15 23:15:44 is older than c

,bucket,keyword,tweet_url,tweet_text,tweet_date
0,Experience,Aflac customer service issues,https://x.com/cnbroussard/status/1882870498834...,"@Entergy\n I'm having customer service issues,...",2025-01-24 19:18:03
1,Experience,Aflac denied my claim,https://x.com/ludovico_carlo5/status/189762692...,@aflac had my Hospital Indemnity claim denied ...,2025-03-06 12:34:50
2,Experience,Aflac denied my claim,https://x.com/ludovico_carlo5/status/189747664...,@aflac\n I was just denied my Hospital indemni...,2025-03-06 02:37:41
3,Experience,Aflac sucks,https://x.com/aflaccck/status/1895165162774253658,i made it bc it was funny but it is an actual ...,2025-02-27 17:32:39
4,Experience,Aflac sucks,https://x.com/LouisianaMimi55/status/188082461...,Saban really kind of sucks in the Aflac commer...,2025-01-19 03:48:27
5,Experience,Aflac sucks,https://x.com/aflaccck/status/1879783939088560521,oh that sucks,2025-01-16 06:53:10
6,Experience,Aflac sucks,https://x.com/TekuaniTruth/status/187708478624...,She sucks the youth out of Ben Aflac and her o...,2025-01-08 20:07:42
7,Experience,Aflac sucks,https://x.com/aflaccck/status/1874868080687616341,do you think they asked her to pose like this ...,2025-01-02 17:19:18
8,Experience,Aflac sucks,https://x.com/l1c4AJD/status/1870239269794841079,Damn this sucks 10 \nYou were my QB from day \...,2024-12-20 22:46:03
9,Experience,Aflac sucks,https://x.com/Richard90618832/status/186416036...,Mets wacko signing 2 yr 34 million for Frankie...,2024-12-04 04:10:41


In [7]:
df_new

,bucket,keyword,tweet_url,tweet_text,tweet_date
0,Experience,Aflac customer service issues,https://x.com/cnbroussard/status/1882870498834...,"@Entergy\n I'm having customer service issues,...",2025-01-24 19:18:03
1,Experience,Aflac denied my claim,https://x.com/ludovico_carlo5/status/189762692...,@aflac had my Hospital Indemnity claim denied ...,2025-03-06 12:34:50
2,Experience,Aflac denied my claim,https://x.com/ludovico_carlo5/status/189747664...,@aflac\n I was just denied my Hospital indemni...,2025-03-06 02:37:41
3,Experience,Aflac sucks,https://x.com/aflaccck/status/1895165162774253658,i made it bc it was funny but it is an actual ...,2025-02-27 17:32:39
4,Experience,Aflac sucks,https://x.com/LouisianaMimi55/status/188082461...,Saban really kind of sucks in the Aflac commer...,2025-01-19 03:48:27
5,Experience,Aflac sucks,https://x.com/aflaccck/status/1879783939088560521,oh that sucks,2025-01-16 06:53:10
6,Experience,Aflac sucks,https://x.com/TekuaniTruth/status/187708478624...,She sucks the youth out of Ben Aflac and her o...,2025-01-08 20:07:42
7,Experience,Aflac sucks,https://x.com/aflaccck/status/1874868080687616341,do you think they asked her to pose like this ...,2025-01-02 17:19:18
8,Experience,Aflac sucks,https://x.com/l1c4AJD/status/1870239269794841079,Damn this sucks 10 \nYou were my QB from day \...,2024-12-20 22:46:03
9,Experience,Aflac sucks,https://x.com/Richard90618832/status/186416036...,Mets wacko signing 2 yr 34 million for Frankie...,2024-12-04 04:10:41


In [21]:
if 'df_new' not in globals():
    df_new = pd.DataFrame(columns=["bucket", "keyword", "tweet_url", "tweet_text", "tweet_date"])

# Combine both datasets
df_combined = pd.concat([df_links, df_new], ignore_index=True)
display(df_combined)
df_combined.to_csv("Twitter_Scraped_Updated.csv", index = False)

,keyword,tweet_url,tweet_text,tweet_date,bucket
0,Aflac company,https://x.com/claudi_surfing/status/1898331370...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 11:14:02,Brand
1,Aflac company,https://x.com/DarthDividend23/status/189821313...,Pretty good start for the month of March with ...,2025-03-08 03:24:13,Brand
2,Aflac company,https://x.com/darioalvez/status/18981822153280...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-08 01:21:21,Brand
3,Aflac company,https://x.com/HIimbelieber2/status/18978253114...,Insurance Giant Aflac Reportedly Sued For Work...,2025-03-07 01:43:08,Brand
4,Aflac company,https://x.com/JulesMoffit/status/1897387492866...,You believe what people are spending on drugs ...,2025-03-05 20:43:24,Brand
...,...,...,...,...,...
871,Aflac experience,https://x.com/aflac_13/status/1878917750611218713,"D2 Transfer Portal LS/Safety\n5’8, 180lbs, 4.0...",2025-01-13 21:31:15,Experience
872,Aflac experience,https://x.com/aflac_13/status/1878917750611218713,"D2 Transfer Portal LS/Safety\n5’8, 180lbs, 4.0...",2025-01-13 21:31:15,Experience
873,Aflac experience,https://x.com/aflac_babiya/status/187022801475...,"When it comes to unexpected medical expenses, ...",2024-12-20 22:01:20,Experience
874,Aflac experience,https://x.com/BigTontoLove/status/186484773183...,"Medicare people, help!\n\nAny experience with ...",2024-12-06 01:42:01,Experience


In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")
chrome_options.add_argument("--headless")  # Runs in background for speed (remove if you want to see the browser)

# Function to generate a random delay between actions
def random_delay():
    return random.uniform(5.5, 15.0)  # Shorter delays for faster scraping

def categorize_keyword(keyword):
    """ Assigns the correct bucket based on the keyword. """
    if keyword in brand_keywords:
        return "Brand"
    elif keyword in experience_keywords:
        return "Experience"
    elif keyword in product_keywords:
        return "Products"
    else:
        return "Unknown"

def login_and_scrape_all(username, password, keywords, cutoff_date_str):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    new_data = []  # List to store new tweets
    cutoff_date = datetime.strptime(cutoff_date_str, "%Y-%m-%d")

    try:
        # Open the login page
        driver.get("https://x.com/login")
        
        # Wait for manual login
        input("Log in manually, then press Enter to start scraping...")

        # Verify login was successful
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='AppTabBar_Home_Link']"))
        )

        # Loop through broad keywords
        for keyword in keywords:
            bucket = categorize_keyword(keyword)
            print(f"Collecting tweets for keyword: {keyword} (Bucket: {bucket})")

            search_url = f"https://x.com/search?q={keyword}&src=typed_query&f=live"
            driver.get(search_url)
            time.sleep(random_delay())  # Shorter delay for speed

            last_height = driver.execute_script("return document.body.scrollHeight")
            stop_scrolling = False

            while not stop_scrolling:
                tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
                for tweet in tweets:
                    try:
                        # Extract tweet URL
                        link_element = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/')]")
                        link = link_element.get_attribute('href')

                        # Extract tweet date
                        date_element = tweet.find_element(By.XPATH, ".//time")
                        tweet_date = datetime.fromisoformat(date_element.get_attribute('datetime')[:-1])

                        # Extract tweet text
                        text_elements = tweet.find_elements(By.XPATH, ".//div[@data-testid='tweetText']")
                        tweet_text = " ".join([elem.text for elem in text_elements]).strip() if text_elements else ""

                        if tweet_date >= cutoff_date:
                            new_data.append((bucket, keyword, link, tweet_text, tweet_date))
                            print(f"Scraped: {link} - {tweet_date} - {bucket}")
                            print(f"Tweet Text: {tweet_text}\n")
                        else:
                            print(f"Tweet from {tweet_date} is older than cutoff. Stopping scrolling.")
                            stop_scrolling = True
                            break  # Stop collecting for this keyword

                    except Exception as e:
                        print(f"Error processing tweet: {e}")
                        continue

                if stop_scrolling:
                    break  # Stop scrolling if tweets are too old

                # Scroll down and wait (optimized scrolling)
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
                time.sleep(random_delay())

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # Stop if no new content loads
                last_height = new_height

    except Exception as e:
        print(f"Error while collecting tweets: {e}")

    finally:
        driver.quit()

    return new_data

# Set up new keyword lists (Non-overlapping)
brand_keywords = [
    "Aflac news", "Aflac sponsorship", "Aflac corporate", "Aflac investment",
    "Aflac CEO", "Aflac stock", "Aflac shareholders", "Aflac employees", 
    "Aflac headquarters", "Aflac advertisement campaign", "Aflac branding", 
    "Aflac media coverage", "Aflac public relations", "Aflac innovation",
    "Aflac press release", "Aflac commercial strategy", "Aflac market expansion"
]

experience_keywords = [
    "Aflac claim wait time", "Aflac reimbursement", "Aflac claim success",
    "Aflac agent experience", "Aflac claim payout", "Aflac phone support",
    "Aflac online claims", "Aflac chat support", "Aflac direct deposit claims",
    "Aflac help center", "Aflac emergency claim", "Aflac application process",
    "Aflac support team", "Aflac policyholder experience", "Aflac customer relations",
    "Aflac premium payment", "Aflac claim processing time", "Aflac call response time"
]

product_keywords = [
    "Aflac disability insurance", "Aflac critical illness", "Aflac vision insurance",
    "Aflac accident coverage", "Aflac family insurance", "Aflac wellness benefits",
    "Aflac prescription coverage", "Aflac hospital coverage", "Aflac maternity benefits",
    "Aflac retirement coverage", "Aflac term life insurance", "Aflac health plan options",
    "Aflac pre-existing conditions", "Aflac travel insurance", "Aflac policy comparison",
    "Aflac cancer treatment benefits", "Aflac injury coverage", "Aflac long-term care"
]

# Combine all new keywords
all_keywords = brand_keywords + experience_keywords + product_keywords

# Set the cutoff date manually (YYYY-MM-DD)
cutoff_date_str = "2024-11-13"  

# Run the scraper for new keywords
df_new = login_and_scrape_all("your_username_here", "your_password_here", all_keywords, cutoff_date_str)

print(f"Scraped {len(df_new)} new tweets across all categories.")


In [23]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")


def random_delay():
    return random.uniform(5.5, 15.0)  # Shorter delays for faster scraping

def categorize_keyword(keyword):
    """ Assigns the correct bucket based on the keyword. """
    if keyword in brand_keywords:
        return "Brand"
    elif keyword in experience_keywords:
        return "Experience"
    elif keyword in product_keywords:
        return "Products"
    else:
        return "Unknown"

def login_and_scrape_all(username, password, keywords, cutoff_date_str):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    new_data = []  # List to store new tweets
    cutoff_date = datetime.strptime(cutoff_date_str, "%Y-%m-%d")

    try:
        # Open the login page
        driver.get("https://x.com/login")
        
        # Wait for manual login
        input("Log in manually, then press Enter to start scraping...")

        # Verify login was successful
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='AppTabBar_Home_Link']"))
        )

        # Loop through broad keywords
        for keyword in keywords:
            bucket = categorize_keyword(keyword)
            print(f"Collecting tweets for keyword: {keyword} (Bucket: {bucket})")

            search_url = f"https://x.com/search?q={keyword}&src=typed_query&f=live"
            driver.get(search_url)
            time.sleep(random_delay())  # Shorter delay for speed

            last_height = driver.execute_script("return document.body.scrollHeight")
            stop_scrolling = False

            while not stop_scrolling:
                tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
                for tweet in tweets:
                    try:
                        # Extract tweet URL
                        link_element = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/')]")
                        link = link_element.get_attribute('href')

                        # Extract tweet date
                        date_element = tweet.find_element(By.XPATH, ".//time")
                        tweet_date = datetime.fromisoformat(date_element.get_attribute('datetime')[:-1])

                        # Extract tweet text
                        text_elements = tweet.find_elements(By.XPATH, ".//div[@data-testid='tweetText']")
                        tweet_text = " ".join([elem.text for elem in text_elements]).strip() if text_elements else ""

                        if tweet_date >= cutoff_date:
                            new_data.append((bucket, keyword, link, tweet_text, tweet_date))
                            print(f"Scraped: {link} - {tweet_date} - {bucket}")
                            print(f"Tweet Text: {tweet_text}\n")
                        else:
                            print(f"Tweet from {tweet_date} is older than cutoff. Stopping scrolling.")
                            stop_scrolling = True
                            break  # Stop collecting for this keyword

                    except Exception as e:
                        print(f"Error processing tweet: {e}")
                        continue

                if stop_scrolling:
                    break  # Stop scrolling if tweets are too old

                # Scroll down and wait (optimized scrolling)
                driver.execute_script("window.scrollBy(0, document.body.scrollHeight / 2);")
                time.sleep(random_delay())

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break  # Stop if no new content loads
                last_height = new_height

    except Exception as e:
        print(f"Error while collecting tweets: {e}")

    finally:
        driver.quit()

    return new_data

# DO NOT CHANGE THESE KEYWORDS - YOUR ORIGINAL KEYWORDS
brand_keywords = [
    "Aflac news", "Aflac sponsorship", "Aflac corporate", "Aflac investment",
    "Aflac CEO", "Aflac stock", "Aflac shareholders", "Aflac employees", 
    "Aflac headquarters", "Aflac advertisement campaign", "Aflac branding", 
    "Aflac media coverage", "Aflac public relations", "Aflac innovation",
    "Aflac press release", "Aflac commercial strategy", "Aflac market expansion"
]

experience_keywords = [
    "Aflac claim wait time", "Aflac reimbursement", "Aflac claim success",
    "Aflac agent experience", "Aflac claim payout", "Aflac phone support",
    "Aflac online claims", "Aflac chat support", "Aflac direct deposit claims",
    "Aflac help center", "Aflac emergency claim", "Aflac application process",
    "Aflac support team", "Aflac policyholder experience", "Aflac customer relations",
    "Aflac premium payment", "Aflac claim processing time", "Aflac call response time"
]

product_keywords = [
    "Aflac disability insurance", "Aflac critical illness", "Aflac vision insurance",
    "Aflac accident coverage", "Aflac family insurance", "Aflac wellness benefits",
    "Aflac prescription coverage", "Aflac hospital coverage", "Aflac maternity benefits",
    "Aflac retirement coverage", "Aflac term life insurance", "Aflac health plan options",
    "Aflac pre-existing conditions", "Aflac travel insurance", "Aflac policy comparison",
    "Aflac cancer treatment benefits", "Aflac injury coverage", "Aflac long-term care"
]

# Combine all new keywords
all_keywords = brand_keywords + experience_keywords + product_keywords

# Set the cutoff date manually (YYYY-MM-DD)
cutoff_date_str = "2024-11-13"  

# Run the scraper for new keywords and store results in a separate DataFrame
df_new_scrape = login_and_scrape_all("your_username_here", "your_password_here", all_keywords, cutoff_date_str)

print(f"Scraped {len(df_new_scrape)} new tweets across all categories.")


Log in manually, then press Enter to start scraping... 


Scraped: https://x.com/knoxnews/status/1895508057011990999 - 2025-02-28 16:15:12 - Brand
Tweet Text: Tennessee will open the 2025 season against Syracuse in the Aflac Kickoff Game at Atlanta's Mercedes-Benz Stadium. Here's how to buy tickets.

Scraped: https://x.com/_diginsurance/status/1895468701173829815 - 2025-02-28 13:38:49 - Brand
Tweet Text: @ZurichNA
 expands middle market business, and more from 
@aflac
, InnSure, 
@Aon_plc
, 
@SapiensIns
, 
@DuckCreekTech
 and 
@safepointins
, in our insurance news roundup. https://bit.ly/4ifCUB5 #insurtech #insurance

Scraped: https://x.com/XVolatility/status/1894481530027713008 - 2025-02-25 20:16:09 - Brand
Tweet Text: Aflac's latest movement can spark some interest! The significant focus in recent news is their partnership with the American Cancer Society, enhancing their market standing as a leader in cancer insurance. Coupling this with their sturdy financial results from Q4, where net

Scraped: https://x.com/aflaccck/status/1893836475034

In [31]:
import pandas as pd

if 'df_new_scrape' not in globals() or not isinstance(df_new_scrape, pd.DataFrame):
    df_new_scrape = pd.DataFrame(df_new_scrape, columns=["bucket", "keyword", "tweet_url", "tweet_text", "tweet_date"])

df_combined = pd.concat([df_combined, df_new_scrape], ignore_index=True)

In [35]:
df_combined.to_csv("Twitter_Scrape_New.csv", index = False)